In [1]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [2]:
%fs ls /FileStore/tables/raw

path,name,size
dbfs:/FileStore/tables/raw/despesas_2009-d0d22.csv,despesas_2009-d0d22.csv,73741287
dbfs:/FileStore/tables/raw/despesas_2010-91775.csv,despesas_2010-91775.csv,96572819
dbfs:/FileStore/tables/raw/despesas_2011-2289d.csv,despesas_2011-2289d.csv,107508369
dbfs:/FileStore/tables/raw/despesas_2012-8ecc3.csv,despesas_2012-8ecc3.csv,99933266
dbfs:/FileStore/tables/raw/despesas_2013-e29e4.csv,despesas_2013-e29e4.csv,108572926
dbfs:/FileStore/tables/raw/despesas_2014-fae0d.csv,despesas_2014-fae0d.csv,104301267
dbfs:/FileStore/tables/raw/despesas_2015-55418.csv,despesas_2015-55418.csv,128248244
dbfs:/FileStore/tables/raw/despesas_2016-414fe.csv,despesas_2016-414fe.csv,122285142
dbfs:/FileStore/tables/raw/despesas_2017-89277.csv,despesas_2017-89277.csv,118080465
dbfs:/FileStore/tables/raw/despesas_2018-2f77b.csv,despesas_2018-2f77b.csv,96715561


In [3]:
# Temas de proposicoes
propositions_theme_schema = StructType([
    StructField('uriProposicao', StringType()),
    StructField('siglaTipo', StringType()),
    StructField('numero', IntegerType()),
    StructField('ano', IntegerType()),
    StructField('codTema', IntegerType()),
    StructField('tema', StringType()),
    StructField('relevancia', IntegerType())
])

df_propositions_theme = spark.read.csv('/FileStore/tables/raw/proposicoesTemas*.csv', header=True, sep=';', schema=propositions_theme_schema)

df_propositions_theme = df_propositions_theme.withColumn('ID_PROPOSICAO', F.split(df_propositions_theme.uriProposicao, '/').getItem(6))

df_propositions_theme = df_propositions_theme.select(
  F.col('ID_PROPOSICAO'),
  F.col('siglaTipo').alias('DC_SIGLA_TIPO'),
  F.col('numero').alias('NU_TEMA_PROPOSICAO'),
  F.col('ano').alias('NU_ANO'),
  F.col('codTema').alias('CD_TEMA'),
  F.col('tema').alias('DC_TEMA'),
  F.col('relevancia').alias('NU_RELEVANCIA')
)

df_propositions_theme = df_propositions_theme.cache()

In [4]:
df_propositions_theme.show(truncate=False)

+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
ID_PROPOSICAO|DC_SIGLA_TIPO|NU_TEMA_PROPOSICAO|NU_ANO|CD_TEMA|DC_TEMA |NU_RELEVANCIA|
+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
2207000 |PL |3379 |2019 |43 |Direito Penal e Processual Penal |0 |
2210208 |EMS |3820 |2019 |64 |Agricultura, Pecuária, Pesca e Extrativismo|0 |
2210208 |EMS |3820 |2019 |58 |Trabalho e Emprego |0 |
2219227 |MSC |410 |2019 |34 |Administração Pública |0 |
2220305 |MSC |418 |2019 |34 |Administração Pública |0 |
2234000 |EMS |4805 |2019 |62 |Ciência, Tecnologia e Inovação |0 |
2234000 |EMS |4805 |2019 |37 |Comunicações |0 |
2234000 |EMS |4805 |2019 |70 |Finanças Públicas e Orçamento |0 |
2234000 |EMS |4805 |2019 |66 |Indústria, Comércio e Serviços |0 |
2236092 |EMS |6606 |2019 |42 |Direito Civil e Processual Civil |0 |
2236092 |EMS |6606 |2019 |66 |Indústria, Comércio e Serviços |0 |
2236041 |EMS |2114 |2019 |57 |Defesa e Segurança |0 |
2236083 |MPV |912 |2019 |44 |Direitos Humanos e Minorias |0 |
2236083 |MPV |912 |2019 |70 |Finanças Públicas e Orçamento |0 |
2236083 |MPV |912 |2019 |55 |Relações Internacionais e Comércio Exterior|0 |
308880 |PL |2441 |2019 |40 |Economia |0 |
317970 |PL |2037 |2019 |34 |Administração Pública |0 |
427339 |PL |2439 |2019 |66 |Indústria, Comércio e Serviços |0 |
501638 |PL |402 |2019 |44 |Direitos Humanos e Minorias |0 |
512431 |PL |2442 |2019 |37 |Comunicações |0 |
+-------------+-------------+------------------+------+-------+-------------------------------------------+-------------+
only showing top 20 rows

In [5]:
# Proposicoes
propositions_schema = StructType([
    StructField('id', IntegerType()),
    StructField('uri', StringType()),
    StructField('siglaTipo', StringType()),
    StructField('numero', IntegerType()),
    StructField('ano', IntegerType()),
    StructField('codTipo', IntegerType()),
    StructField('descricaoTipo', StringType()),
    StructField('ementa', StringType()),
    StructField('ementaDetalhada', StringType()),
    StructField('keywords', StringType()),
    StructField('dataApresentacao', StringType()),
    StructField('uriOrgaoNumerador', StringType()),
    StructField('uriPropAnterior', StringType()),
    StructField('uriPropPrincipal', StringType()),
    StructField('uriPropPosterior', StringType()),
    StructField('urlInteiroTeor', StringType()),
    StructField('urnFinal', StringType()),
    StructField('ultimoStatus_dataHora', StringType()),
    StructField('ultimoStatus_sequencia', IntegerType()),
    StructField('ultimoStatus_uriRelator', StringType()),
    StructField('ultimoStatus_idOrgao', IntegerType()),
    StructField('ultimoStatus_siglaOrgao', StringType()),
    StructField('ultimoStatus_uriOrgao', StringType()),
    StructField('ultimoStatus_regime', StringType()),
    StructField('ultimoStatus_descricaoTramitacao', StringType()),
    StructField('ultimoStatus_idTipoTramitacao', IntegerType()),
    StructField('ultimoStatus_descricaoSituacao', StringType()),
    StructField('ultimoStatus_idSituacao', IntegerType()),
    StructField('ultimoStatus_despacho', StringType()),
    StructField('ultimoStatus_url', StringType())
])

df_propositions = spark.read.csv('/FileStore/tables/raw/proposicoes_*.csv', header=True, sep=';', schema=propositions_schema)

df_propositions = df_propositions.cache()

df_propositions = df_propositions.select(
    F.col('id').alias('ID_PROPOSICAO'),
    F.col('siglaTipo').alias('SG_TIPO'),
    F.col('numero').alias('NU_PROPOSICAO'),
    F.col('ano').alias('NU_ANO'),
    F.col('codTipo').alias('CD_TIPO'),
    F.col('descricaoTipo').alias('DC_TIPO'),
    F.col('ementa').alias('DC_EMENTA'),
    F.col('ementaDetalhada').alias('DC_EMENTA_DETALHADA'),
    F.col('keywords').alias('DC_KEYWORDS'),
    F.col('dataApresentacao').alias('DT_APRESENTACAO')
)

In [6]:
display(df_propositions)

ID_PROPOSICAO,SG_TIPO,NU_PROPOSICAO,NU_ANO,CD_TIPO,DC_TIPO,DC_EMENTA,DC_EMENTA_DETALHADA,DC_KEYWORDS,DT_APRESENTACAO
865934,MPV,667,2015,291,Medida Provisória,"Abre crédito extraordinário, em favor dos órgãos e empresas estatais, constantes do Projeto de Lei Orçamentária de 2015, no valor de R$ 74.014.218.398,00, para os fins que especifica.",null,"Crédito extraordinário, órgão público, Orçamento Fiscal, Seguridade social, empresa estatal, Orçamento de Investimentos.",2015-01-05T13:07:00
865943,PRL,3,0,190,Parecer do Relator,"Parecer da Relatora, Dep. Maria Lucia Prandi (PT-SP), pela aprovação deste, e do PL 7808/2014, apensado, com substitutivo.",null,null,2015-01-08T11:35:00
865944,SBT,1,0,255,Substitutivo,"Acresce inciso ao parágrafo único do art. 13 da Lei nº 9.795, de 27 de abril de 1999, para prever a coleta seletiva de lixo eletrônico de pequeno porte nas escolas.",null,null,2015-01-08T11:37:00
865949,PRL,1,0,190,Parecer do Relator,"Parecer do Relator, Dep. Alfredo Sirkis (PSB-RJ), pela rejeição deste, e do PL 3846/2012, apensado.",null,null,2015-01-12T10:13:00
865950,PRL,1,0,190,Parecer do Relator,"Parecer do Relator, Dep. Alfredo Sirkis (PSB-RJ), pela aprovação, com substitutivo.",null,null,2015-01-12T10:14:00
865951,SBT,1,0,255,Substitutivo,"Estabelece a Política Nacional de Captação, Armazenamento e Aproveitamento de Águas Pluviais e define normas gerais para sua promoção.",null,null,2015-01-12T10:20:00
865953,PRL,1,0,190,Parecer do Relator,"Parecer do Relator, Dep. Ronaldo Caiado (DEM-GO), pela rejeição.",null,null,2015-01-12T11:22:00
865959,PL,8328,2015,139,Projeto de Lei,"Altera a Lei n° 10.098, de 19 de dezembro de 2000, para determinar a reserva de assentos para pessoas com deficiência em locais de espetáculos, conferências, aulas e outros de natureza similar.","""Substitui a expressão """"pessoa portadora de deficiência"""" pela expressão """"pessoa com deficiência"""".""","Alteração, Lei da Acessibilidade, pessoa com deficiência auditiva, pessoa com deficiência visual, reserva, assento preferencial, evento público, substituição, expressão vocabular, pessoa com deficiência.",2015-01-09T17:43:00
865960,PL,8329,2015,139,Projeto de Lei,"Altera a Lei n° 10.741, de 1° de outubro de 2003 (Estatuto do Idoso), para estabelecer gratuidade de acesso do idoso a eventos esportivos realizados em estádios e ginásios. bem como a museus mantidos com verbas públicas e a eventos culturais patrocinados pelo Governo Federal.",null,"Alteração, Estatuto do Idoso, gratuidade, idoso, acesso, evento esportivo, museu, evento cultural, patrocínio, governo federal.",2015-01-09T17:43:00
865961,PL,8330,2015,139,Projeto de Lei,"""Altera a Lei n° 11.340, de 7 de agosto de 2006 (Lei Maria da Penha). para garantir à mulher vítima de violência doméstica o recebimento de beneficio eventual, e a Lei n° 8.742, de 7 de dezembro de 1993 (Lei Orgânica da Assistência Social), para definir o termo """"situação de vulnerabilidade temporária"""" de que trata o seu art. 22"""".""",null,"Alteração, Lei Maria da Penha, garantia, benefício eventual, mulher, vítima, violência doméstica e familiar, profilaxia, doença sexualmente transmissível (DST), Aids, definição, situação de vulnerabilidade temporária.",2015-01-09T17:43:00


In [7]:
# Despesas
expenses_schema = StructType([
    StructField('txtNomeParlamentar', StringType()),
    StructField('cpf', StringType()),
    StructField('ideCadastro', StringType()),
    StructField('nuCarteiraParlamentar', StringType()),
    StructField('nuLegislatura', IntegerType()),
    StructField('sgUF', StringType()),
    StructField('sgPartido', StringType()),
    StructField('codLegislatura', StringType()),
    StructField('numSubCota', IntegerType()),
    StructField('txtDescricao', StringType()),
    StructField('numEspecificacaoSubCota', IntegerType()),
    StructField('txtDescricaoEspecificacao', StringType()),
    StructField('txtFornecedor', StringType()),
    StructField('txtCNPJCPF', StringType()),
    StructField('txtNumero', StringType()),
    StructField('indTipoDocumento', StringType()),
    StructField('datEmissao', StringType()),
    StructField('vlrDocumento', DoubleType()),
    StructField('vlrGlosa', DoubleType()),
    StructField('vlrLiquido', DoubleType()),
    StructField('numMes', IntegerType()),
    StructField('numAno', IntegerType()),
    StructField('numParcela', IntegerType()),
    StructField('txtPassageiro', StringType()), 
    StructField('txtTrecho', StringType()), 
    StructField('numLote', IntegerType()), 
    StructField('numRessarcimento', IntegerType()), 
    StructField('vlrRestituicao', DoubleType()), 
    StructField('nuDeputadoId', IntegerType()), 
    StructField('ideDocumento', IntegerType()), 
    StructField('urlDocumento', StringType())
])

df_expenses = spark.read.csv('/FileStore/tables/raw/despesas*.csv', header=True, sep=';', schema=expenses_schema)

df_expenses = df_expenses.where(F.col('ideCadastro').isNotNull())

df_expenses = df_expenses.cache()

df_politicians = df_expenses.select(
  F.col('ideCadastro').alias('ID_DEPUTADO'),
  F.col('txtNomeParlamentar').alias('NM_PARLAMENTAR'),
  F.col('cpf').alias('DC_CPF_PARLAMENTAR'),
  F.col('nuCarteiraParlamentar').alias('NU_CARTEIRA_PARLAMENTAR'),
  F.col('nuLegislatura').alias('NU_LEGISLATURA'),
  F.col('sgUF').alias('SG_UF'),
  F.col('sgPartido').alias('SG_PARTIDO'),
  F.col('codLegislatura').alias('CD_LEGISLATURA')
).drop_duplicates()

df_expenses = df_expenses.select(
  F.col('ideCadastro').alias('ID_DEPUTADO'),
  F.col('numSubCota').alias('NU_SUBCOTA'),
  F.col('txtDescricao').alias('DC_DESCRICAO'),
  F.col('numEspecificacaoSubCota').alias('NU_ESPECIFICACAO_SUBCOTA'),
  F.col('txtDescricaoEspecificacao').alias('DC_ESPECIFICACAO'),
  F.col('txtFornecedor').alias('NM_FORNECEDOR'),
  F.col('txtCNPJCPF').alias('DC_CPFCNPJ_FORNECEDOR'),
  F.col('txtNumero').alias('DC_NUMERO'),
  F.col('indTipoDocumento').alias('DC_TIPO_DOCUMENTO'),
  F.col('datEmissao').alias('DT_EMISSAO'),
  F.col('vlrDocumento').alias('VL_DOCUMENTO'),
  F.col('vlrGlosa').alias('VL_GLOSA'),
  F.col('vlrLiquido').alias('VL_LIQUIDO'),
  F.col('numMes').alias('NU_MES'),
  F.col('numAno').alias('NU_ANO'),
  F.col('numParcela').alias('NU_PARCELA'),
  F.col('txtPassageiro').alias('DC_PASSAGEIRO'), 
  F.col('txtTrecho').alias('DC_TRECHO'), 
  F.col('numLote').alias('NU_LOTE'), 
  F.col('numRessarcimento').alias('NU_RESSARCIMENTO'), 
  F.col('vlrRestituicao').alias('VL_RESTITUICAO'), 
  F.col('ideDocumento').alias('ID_DOCUMENTO')
)

In [8]:
display(df_politicians)

ID_DEPUTADO,NM_PARLAMENTAR,DC_CPF_PARLAMENTAR,NU_CARTEIRA_PARLAMENTAR,NU_LEGISLATURA,SG_UF,SG_PARTIDO,CD_LEGISLATURA
74037,FÁTIMA BEZERRA,16025733449,121,2011,RN,PT,54
178888,VICTOR MENDES,80897460391,82,2015,MA,MDB,55
178928,CHRISTIANE DE SOUZA YARED,49156470991,442,2015,PR,PR,55
74048,RAUL JUNGMANN,24444928468,541,2007,PE,PPS,53
74075,ELCIONE BARBALHO,00605387249,21,2007,PA,PMDB,53
141443,GERMANO BONOW,00741108291,489,2007,RS,DEM,53
141460,JOÃO OLIVEIRA,08865817100,61,2007,TO,DEM,53
73909,OSVALDO BIOLCHI,04962796020,578,2007,RS,PMDB,53
74337,MILTON BARBOSA,06271138500,600,2007,BA,PSC,53
81297,DRA. SORAYA MANATO,null,277,2015,ES,PSL,55


In [9]:
display(df_expenses)

ID_DEPUTADO,NU_SUBCOTA,DC_DESCRICAO,NU_ESPECIFICACAO_SUBCOTA,DC_ESPECIFICACAO,NM_FORNECEDOR,DC_CPFCNPJ_FORNECEDOR,DC_NUMERO,DC_TIPO_DOCUMENTO,DT_EMISSAO,VL_DOCUMENTO,VL_GLOSA,VL_LIQUIDO,NU_MES,NU_ANO,NU_PARCELA,DC_PASSAGEIRO,DC_TRECHO,NU_LOTE,NU_RESSARCIMENTO,VL_RESTITUICAO,ID_DOCUMENTO
3151,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,0,null,CEMIG DISTRIBUIÇAO S.A.,069.811.800/0011-6,001562795,0,2015-01-13T00:00:00,13.19,0.25,12.94,1,2015,0,null,null,1169573,null,null,5611810
3151,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,0,null,CEMIG DISTRIBUIÇAO S.A.,069.811.800/0011-6,001562796,0,2015-01-13T00:00:00,90.43,1.3,89.13,1,2015,0,null,null,1163724,null,null,5591461
3151,1,MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR,0,null,COPASA,172.811.060/0010-3,00115027414617,0,2015-01-14T00:00:00,57.8,0.99,56.81,1,2015,0,null,null,1163724,null,null,5591446
3151,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,CASCOL COMBUSTÍVEIS PARA VEÍCULOS LTDA.,003.065.970/0630-8,014297,0,2015-01-30T00:00:00,162.84,0.0,162.84,1,2015,0,null,null,1169573,null,null,5611848
3151,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,J P DERIVADOS DE PETROLEO LTDA,029.984.140/0018-7,041748,0,2015-01-29T00:00:00,100.01,0.0,100.01,1,2015,0,null,null,1169573,null,null,5611851
3151,3,COMBUSTÍVEIS E LUBRIFICANTES.,1,Veículos Automotores,POSTO REM LTDA,417.052.110/0010-7,471379,0,2015-01-28T00:00:00,210.51,0.0,210.51,1,2015,0,null,null,1169574,null,null,5611844
3151,4,"CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.",0,null,MENEZES - CONSULTORES E ADVOGADOS ASSOCIADOS,072.683.080/0016-1,201500000000192,0,2015-02-05T00:00:00,9500.0,0.0,9500.0,1,2015,0,null,null,1169574,null,null,5611920
3151,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,0,null,ZOOM PRODUTORA LTDA,028.539.450/0018-1,201500000000037,0,2015-01-30T00:00:00,9000.0,0.0,9000.0,1,2015,0,null,null,1169574,null,null,5611918
3151,8,SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.,0,null,PROTEL,429.396.450/0013-4,201500000024718,0,2015-01-07T00:00:00,243.0,0.0,243.0,1,2015,0,null,null,1163725,null,null,5591429
3151,10,TELEFONIA,0,null,CELULAR FUNCIONAL,000.000.000/0000-1,5882419,0,2015-01-13T00:00:00,39.24,0.0,39.24,1,2015,0,null,null,0,0,null,0


In [10]:
# Autores das Proposicoes
propositions_authors_schema = StructType([
    StructField('idProposicao', IntegerType()),
    StructField('uriProposicao', StringType()),
    StructField('idDeputadoAutor', IntegerType()),
    StructField('uriAutor', StringType()),
    StructField('codTipoAutor', StringType()),
    StructField('tipoAutor', StringType()),
    StructField('nomeAutor', StringType()),
    StructField('siglaPartidoAutor', StringType()),
    StructField('uriPartidoAutor', StringType()),
    StructField('siglaUFAutor', StringType()),
    StructField('ordemAssinatura', IntegerType()),
    StructField('proponente', IntegerType())
])

df_propositions_authors = spark.read.csv('/FileStore/tables/raw/proposicoesAutores*.csv', header=True, sep=';', schema=propositions_authors_schema)

df_propositions_authors = df_propositions_authors.cache()

df_propositions_authors = df_propositions_authors.select(
    F.col('idProposicao').alias('ID_PROPOSICAO'),
    F.col('idDeputadoAutor').alias('ID_DEPUTADO'),
    F.col('codTipoAutor').alias('CD_TIPO_AUTOR'),
    F.col('tipoAutor').alias('TP_AUTOR'),
    F.col('nomeAutor').alias('NM_AUTOR'),
    F.col('siglaPartidoAutor').alias('SG_PARTIDO_AUTOR'),
    F.col('siglaUFAutor').alias('SG_UF_AUTOR'),
    F.col('ordemAssinatura').alias('NU_ORDEM_ASSINATURA'),
    F.col('proponente').alias('NU_PROPONENTE')
)

df_propositions_authors = df_propositions_authors.where(F.col('ID_DEPUTADO').isNotNull())

In [11]:
display(df_propositions_authors)

ID_PROPOSICAO,ID_DEPUTADO,CD_TIPO_AUTOR,TP_AUTOR,NM_AUTOR,SG_PARTIDO_AUTOR,SG_UF_AUTOR,NU_ORDEM_ASSINATURA,NU_PROPONENTE
2237302,141450,10000,Deputado,Hugo Leal,PSD,RJ,1,1
2237301,141450,10000,Deputado,Hugo Leal,PSD,RJ,1,1
2237300,141450,10000,Deputado,Hugo Leal,PSD,RJ,1,1
2237299,204521,10000,Deputado,Abou Anni,PSL,SP,1,1
2237298,204375,10000,Deputado,Leda Sadala,AVANTE,AP,1,1
2237297,204521,10000,Deputado,Abou Anni,PSL,SP,1,1
2237296,204521,10000,Deputado,Abou Anni,PSL,SP,1,1
2237295,204375,10000,Deputado,Leda Sadala,AVANTE,AP,1,1
2237294,204521,10000,Deputado,Abou Anni,PSL,SP,1,1
2237293,141411,10000,Deputado,Dagoberto Nogueira,PDT,MS,1,1


In [12]:
# Export Data to curated zone

df_propositions_theme.write.parquet('/FileStore/tables/curated/propositions_themes')
df_propositions_authors.write.parquet('/FileStore/tables/curated/propositions_authors')
df_propositions.write.parquet('/FileStore/tables/curated/propositions')
df_politicians.write.parquet('/FileStore/tables/curated/politicians')
df_expenses.write.parquet('/FileStore/tables/curated/expenses')